In [1]:
import pandas as pd
import src.scripts.artemis as artemis

In [2]:
def replace_quotes(val):
    return val.replace('"', '')

m_df = pd.read_csv("../data/external/threatfox_malicious_domains.csv")
m_df['domain'] = m_df.ioc_value.apply(replace_quotes)
m_df['malicious'] = 1

In [3]:
b_df = pd.read_csv("../data/raw/majestic_million_small.csv")
b_df = b_df.rename(columns={'Domain':'domain'})
b_df['malicious'] = 0

In [4]:
cols = ['domain', 'malicious']
domains_df = pd.concat([b_df[cols], m_df[cols]], ignore_index=True)

In [5]:
domains_df

,domain,malicious
0,google.com,0
1,facebook.com,0
2,youtube.com,0
3,twitter.com,0
4,instagram.com,0
...,...,...
16866,retechnolodgy.top,1
16867,rpoznahu.top,1
16868,aluditos.top,1
16869,cosmokosmo.best,1


In [6]:
test_df = domains_df.sample(100, random_state=0)

In [7]:
test_df.head()

,domain,malicious
10685,www.myserv012.com,1
6298,wthr.com,0
10117,vitanoise.com,1
8710,nctu.edu.tw,0
12190,mezugen.com,1


In [8]:
test_df.malicious.value_counts()

0    64
1    36
Name: malicious, dtype: int64

In [9]:
domains = test_df.domain.tolist()
y = test_df.malicious.tolist()
zipped = zip(domains, y)

In [10]:
results = []
for domain, y in zipped:
    prediction = artemis.predict(domain)
    results.append(prediction)
    sleep(0.5)
test_df['prediction'] = results

NameError: name 'sleep' is not defined

In [ ]:
test_df